In [ ]:

# low vol strategy
from quantopian.research import run_pipeline, prices
from quantopian.research.experimental import history
import numpy as np
import pandas as pd
from quantopian.pipeline import Pipeline
from quantopian.pipeline import CustomFactor
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.filters import Q500US,Q3000US,Q1500US
from quantopian.pipeline.data import Fundamentals

nq=5
nq_vol=3
nq_gp=3
nq_pe=3
nq_ps=3
nq_pb=3
trade_count = 3
tier = 0
is_trading_day = False
max_leverage = [0]
portfolio_mom_day_count = 5
is_aggressive_mode = False
portfolio_percent = {}
    
# 6 days before
start_date = '2020-10-21'
end_date = '2020-10-28'

SPY_start_date = '2019-10-28'

SPY = [symbols('SPY')]
long_leverage = 1.0
short_leverage = -0.7

replace_prices ={ 'NET':54.72, 'CL':77.53, 'ON':24.35, 'A':103.52, 'CMCS_A':42.48, 'VIA_B':28, 'GOOG_L':1520.1, 'DISC_A':19.7, 'AAPL':112.56, 'ABMD':284.31, 'ABT':106.44, 'ADSK':239.5, 'ADBE':461.12, 'ADI':116.23, 'ADM':49.74, 'AEP':92.76, 'AFL':34.58, 'HES':35.85, 'AIG':29.95, 'ALK':35.26, 'AMAT':57.06, 'AMD':76.15, 'AME':99.91, 'AMGN':217.36, 'AON':193.41, 'APA':7.84, 'APD':276.56, 'APH':112, 'ADP':155.17, 'AXP':91.39, 'AZO':1115.2, 'BA':149.9, 'BAC':23.19, 'BAX':79, 'BBBY':21.18, 'BBY':111.34, 'BDX':237.2, 'BK':34.18, 'BLL':89.71, 'BMY':57.89, 'BSX':35.05, 'CAG':36.22, 'CAT':150.98, 'C':40.91, 'CAH':46.57, 'CDNS':106.13, 'CERN':71.94, 'CHD':90.57, 'CI':171.27, 'CLX':212.24, 'CMS':65.2, 'COG':18.44, 'COST':367.96, 'CPB':47.6, 'CSCO':35.79, 'CSX':76.4, 'CTAS':318.49, 'LUMN':8.69, 'CMI':213.68, 'D':80.91, 'DE':223.41, 'DHR':235.86, 'DIS':119.56, 'DHI':69.26, 'DTE':124.57, 'DUK':92.52, 'DVN':8.12, 'ECL':185.82, 'ED':81.62, 'EFX':143.37, 'EMR':64.75, 'EOG':32.21, 'EA':125.34, 'ETN':102.23, 'ETR':105.63, 'F':7.66, 'FAST':42.87, 'M':6.08, 'FDX':263.06, 'FISV':98.06, 'FITB':22.42, 'NEE':75.26, 'GD':134.24, 'GE':7.7, 'INO':10.49, 'GILD':58.21, 'GIS':60.23, 'GLW':31.91, 'GPS':19.56, 'HAL':11.59, 'MNST':77.96, 'HBAN':9.72, 'WELL':52.59, 'PEAK':26.84, 'HD':270, 'HOLX':70.88, 'HRL':49.45, 'LHX':163.61, 'HSY':140.06, 'HUM':420.16, 'HPQ':17.43, 'IBM':107.11, 'BIIB':249.31, 'IDXX':436.95, 'IFF':102.35, 'IMMU':87.86, 'INTC':44.48, 'IP':43.13, 'TT':128.14, 'ITW':194.42, 'JNJ':138.82, 'K':64.28, 'KEY':12.3, 'KLAC':192.23, 'KMB':132.24, 'KO':48.21, 'KR':32.07, 'KSS':21.07, 'KSU':172.72, 'LEN':72.86, 'LLY':132.5, 'LOW':161.94, 'LRCX':334.35, 'LB':32.41, 'LUV':37.71, 'MAS':54.74, 'MKC':189.21, 'MCD':216.92, 'MDT':102.3, 'CVS':56.9, 'MGM':20.07, 'SPGI':323.99, 'MMC':107.69, 'MMM':159.11, 'MO':36.48, 'MSI':167.19, 'MRK':76.38, 'MRO':3.86, 'MSFT':204.93, 'MU':49.72, 'MXIM':68.52, 'MYL':14.09, 'NBL':8.46, 'NEM':58.64, 'NKE':123.88, 'JWN':12.35, 'NOC':300.35, 'NSC':197.67, 'ES':90.88, 'ODFL':185.84, 'OKE':29.07, 'OMC':47.01, 'ORCL':55.73, 'OXY':8.88, 'PAYX':80.47, 'PCAR':86.07, 'PCG':9.81, 'PEG':59.87, 'PEP':135.67, 'PFE':36.13, 'PG':139.01, 'PGR':91.96, 'PH':205.23, 'PHM':41.71, 'PNC':107.89, 'PPG':125.66, 'PPL':28.17, 'QCOM':122.32, 'QDEL':270, 'RAD':9.53, 'REGN':566.97, 'ROK':231.64, 'ROP':373.78, 'ROST':87.77, 'T':26.61, 'SBUX':87.2, 'SCHW':37.48, 'SHW':669.9, 'SLB':14.44, 'SNPS':216.79, 'SO':59.42, 'TRV':120.4, 'STT':59.53, 'SYK':207.7, 'SWK':164.65, 'NLOK':20.42, 'SYY':55.82, 'TER':85.7, 'TIF':128.09, 'TJX':51.92, 'TMO':484.36, 'TROW':137.11, 'TXN':142.67, 'JCI':41.42, 'TSN':56.88, 'UNH':311.97, 'UNP':173.87, 'RTX':53.76, 'VAR':172.69, 'VFC':69.68, 'VIAC':28.05, 'VLO':36.11, 'VMC':136.79, 'VRTX':210.46, 'WBA':35.06, 'WDC':38.81, 'WEC':101.56, 'WFC':20.91, 'WHR':191.65, 'WMB':18.84, 'WMT':140.75, 'WY':27.34, 'X':9.63, 'XLNX':119.91, 'XOM':31.72, 'FL':37.09, 'CB':126.84, 'INTU':319.35, 'MCHP':103.9, 'ORLY':442.84, 'RCL':54.45, 'BWA':36.17, 'EQR':46.87, 'ATVI':79.76, 'HST':10.44, 'INCY':85.86, 'SPG':60.6, 'MLM':247.46, 'TSCO':134.18, 'ALB':90.84, 'CPRT':109.95, 'PENN':57.16, 'SIRI':5.79, 'O':58.06, 'COF':72.08, 'MCK':147.29, 'DLTR':91.8, 'LMT':357.46, 'DRI':91.17, 'LH':205.12, 'FCX':16.77, 'EL':222.71, 'NTAP':42.3, 'CTXS':116.37, 'HIG':36.86, 'NVAX':84.5, 'VXRT':5.05, 'ALXN':116.39, 'EIX':58.81, 'CHKP':116.7, 'ETFC':49.26, 'DGX':126.63, 'CIEN':38.6, 'KMX':86.3, 'AMTD':39.97, 'TTWO':164.41, 'AMZN':3196, 'ARE':153.61, 'TFC':40.86, 'SRPT':138.36, 'BXP':72.26, 'MS':47.54, 'PXD':82.3, 'CHRW':91.44, 'NLY':6.99, 'YUM':95.85, 'FE':32.77, 'CAR':34.04, 'URI':168.16, 'VTR':39.28, 'AVB':135.03, 'CTSH':71.59, 'WM':108, 'CCI':157.96, 'NVDA':514.62, 'BKNG':1638.6, 'GS':190.83, 'SBAC':297.15, 'BMRN':74.97, 'CSGP':789.68, 'AKAM':99.01, 'BLK':599.07, 'PLUG':14.69, 'UPS':162.38, 'TGT':155.22, 'EW':74.36, 'MET':36.6, 'MRVL':39.79, 'ILMN':314.75, 'VZ':56.77, 'XEL':71.81, 'LNG':46.55, 'TPR':20.91, 'EXC':40.36, 'MCO':259.38, 'ALGN':449.99, 'EXAS':129, 'GPN':162.37, 'SGEN':195.64, 'MDLZ':53.88, 'WLTW':194.26, 'FIS':134.84, 'ABC':96.95, 'ZBH':134.08, 'ANTM':291.39, 'CVX':66.59, 'AAP':147.62, 'CNC':61.95, 'PRU':61.51, 'JBLU':11.21, 'NFLX':493.94, 'OSTK':70.8, 'SWKS':140, 'COP':28.81, 'CNP':21.72, 'DKS':57.05, 'WYNN':68.3, 'CME':149.16, 'EQIX':764.19, 'STX':47.24, 'CCL':12.71, 'AMT':232.78, 'SRE':127.95, 'AEE':83.11, 'PLD':98.3, 'EBAY':52.67, 'ALL':87.53, 'STZ':166.09, 'PSA':232.66, 'JPM':96.53, 'USB':37.35, 'HON':162.7, 'ISRG':695.02, 'ACN':213.64, 'MAR':89.27, 'XPO':89.6, 'CRM':240.07, 'DPZ':388.41, 'DLR':149.5, 'LVS':46.76, 'DXCM':345.38, 'EXPE':91.48, 'ICE':94.17, 'UAA':13.32, 'LYV':49.94, 'CMG':1265.01, 'UAL':32.36, 'TDG':462.31, 'MA':297.05, 'HBI':16.7, 'WU':20.34, 'SPR':18.21, 'SRNE':7.17, 'TMUS':110.71, 'DAL':28.85, 'DFS':62.55, 'TEL':98.03, 'BX':49.71, 'LULU':328.3, 'CXO':41.52, 'MELI':1262.71, 'VMW':132.66, 'RF':12.66, 'ULTA':205.73, 'PM':69.77, 'V':180.57, 'AWK':154.71, 'AGNC':14.09, 'WKHS':16.5, 'AVGO':347.21, 'VRSK':180.45, 'DG':213.14, 'FTNT':119.93, 'CHTR':582.34, 'LYB':70.31, 'TSLA':412.12, 'KKR':34.59, 'NXPI':127.86, 'GM':33.86, 'FLT':225.12, 'KMI':11.87, 'HCA':127.77, 'YNDX':57.93, 'SAVE':15.62, 'MPC':27.03, 'HZNP':75.71, 'APTV':92.9, 'ZNGA':9.34, 'ENPH':105.55, 'PSX':44.67, 'SPLK':204, 'FB':271.83, 'NOW':483.63, 'FIVE':132.25, 'PANW':227.12, 'WDAY':209.28, 'FANG':24.88, 'RH':357.95, 'ABBV':80.55, 'NCLH':15.22, 'ZTS':159.54, 'IQV':156.73, 'CDW':118.95, 'KODK':7.44, 'RNG':272.05, 'BURL':194.88, 'VEEV':292.3, 'WIX':261.59, 'TWTR':49.66, 'CHGG':76.01, 'TNDM':112.25, 'AAL':10.7, 'HLT':84.44, 'ALLY':26.49, 'GRUB':75.59, 'PAYC':371.12, 'ZEN':104.8, 'PE':10.18, 'ANET':205.03, 'INFO':79.61, 'SYF':24.97, 'CZR':45.96, 'CFG':25.92, 'W':279.39, 'HUBS':296.65, 'PLAY':16.94, 'KEYS':103.92, 'QRVO':126, 'SHAK':67.77, 'SEDG':265.86, 'GDDY':70.24, 'ETSY':143.41, 'TRU':78.93, 'TDOC':226.29, 'KHC':29.74, 'PYPL':194.96, 'OLLI':91.55, 'LITE':81.21, 'RUN':55.35, 'PLNT':60.78, 'HPE':8.45, 'SQ':167.02, 'TEAM':209.5, 'FTV':61.26, 'TWLO':280.08, 'TTD':594, 'COUP':285.79, 'YUMC':52.59, 'AA':12.43, 'INVH':27.7, 'AYX':137.01, 'OKTA':218.58, 'CVNA':200.27, 'IR':35.22, 'ATUS':26.64, 'BKR':13.83, 'DD':55.97, 'ROKU':219.85, 'SPCE':17.53, 'MDB':237.56, 'VICI':22.39, 'ZS':144.2, 'DBX':18.5, 'SPOT':279.42, 'DOCU':219.05, 'CDAY':87.14, 'EVRG':55.92, 'NKLA':19.74, 'ELAN':31.22, 'PLAN':56.51, 'STNE':53.63, 'LIN':25, 'DELL':61.28, 'MRNA':0.83, 'FOXA':49.69, 'LYFT':21.5, 'PINS':49.73, 'ZM':522.53, 'SHLL':20.15, 'BYND':156.6, 'UBER':33.23, 'FSLY':71.75, 'CTVA':31.75, 'CRWD':131.08, 'CHWY':72.39, 'WORK':26.7, 'DKNG':38.97, 'LVGO':144.99, 'DT':38.43, 'DDOG':98.86, 'PTON':122.57, 'BILL':107.2, 'OTIS':59.45, 'CARR':33.15, 'MTCH':121.45, 'RKT':61.97, 'SPY':328.81}

def my_pipeline():
    """
    The original algorithm used the following filters:
        1. common stock
        2 & 3. not limited partnership - name and database check
        4. database has fundamental data
        5. not over the counter
        6. not when issued
        7. not depository receipts
        8. primary share
        9. high dollar volume
    Check Scott's notebook for more details.
    
    This updated version uses Q1500US, one of the pipeline's built-in base universes. 
    Lesson 11 from the Pipeline tutorial offers a great overview of using multiple 
    filters vs using the built-in base universes:
    https://www.quantopian.com/tutorials/pipeline#lesson11
    
    More detail on the selection criteria of this filter can be found  here:
    https://www.quantopian.com/posts/the-q500us-and-q1500us 
    """
    base_universe = Q500US() #& (is_consumer_cyclical | is_technology)
    total_assets = Fundamentals.total_assets.latest
    gross_profit = Fundamentals.gross_profit.latest
    net_income = Fundamentals.net_income_income_statement.latest
    market_cap = Fundamentals.market_cap.latest
    roe = Fundamentals.roe.latest
    #ipo_date = Fundamentals.ipo_date.latest
    pe_ratio = Fundamentals.pe_ratio.latest
    pb_ratio = Fundamentals.pb_ratio.latest
    #ps_ratio = Fundamentals.ps_ratio.latestS
    
    #market_cap = Fundamentals.market_cap.latest
    #roe = Fundamentals.roe.latest
    # The example algorithm - mean reversion. Note the tradable filter used as a mask.
     
    my_pipe = Pipeline(
              columns={
                'GP': gross_profit / total_assets,
                #'rel_diff': rel_diff
                'pe': pe_ratio,
                'pb': pb_ratio,
                #'ps': ps_ratio,
                'total_assets': total_assets,
                'gross_profit': gross_profit,
                'net_income': net_income,
                'market_cap': market_cap,
                #'ipo_date': ipo_date
                #'GM': gross_profit / market_cap
                #'roe': roe
              },
              screen = base_universe
          )
    #bottom_rel_diff = rel_diff.bottom(250)
    #my_pipe.add(bottom_rel_diff, 'bottom_rel_diff')
     
    return my_pipe

pipe=my_pipeline()
result = run_pipeline(pipe, end_date, end_date)
result = result.reset_index(level=0)
output = result.dropna()
security_set = output.index.tolist()
Universe500=list(security_set)
Universe500_symbols = [sec.symbol for sec in Universe500]
    #Universe500=context.output.index.tolist()
prices = history(Universe500,'price',start_date,end_date,'daily')

for col in prices.columns:
    if col.symbol in replace_prices:
        prices[col][-1] = replace_prices[col.symbol]
        
#P = prices(Universe500, '2018-10-25','2018-11-1')
daily_rets=np.log(prices/prices.shift(1))
rets=(prices.iloc[-2] - prices.iloc[0]) / prices.iloc[0]
    # If you don't want to skip the most recent return, however, use .iloc[-1] instead of .iloc[-2]:
    # rets=(prices.iloc[-1] - prices.iloc[0]) / prices.iloc[0]
stdevs=daily_rets.std(axis=0)
rets_df=pd.DataFrame(rets,columns=['five_day_ret'])
stdevs_df=pd.DataFrame(stdevs,columns=['stdev_ret'])
output=output.join(rets_df,how='outer')
output=output.join(stdevs_df,how='outer')
output['ret_quantile']=pd.qcut(output['five_day_ret'],nq,labels=False)+1
output['stdev_quantile']=pd.qcut(output['stdev_ret'],3,labels=False)+1
output['gp_quantile']=pd.qcut(output['GP'],nq_gp,labels=False)+1
output['ret_num']=output['five_day_ret'].rank(ascending=True,method='first')
output['stdev_num']=output['stdev_ret'].rank(ascending=False,method='first')
output['gp_num']=output['GP'].rank(ascending=True,method='first')
output['pb_quantile']=pd.qcut(output['pb'],nq_pb,labels=False)+1

pri = history(SPY,'price',SPY_start_date,end_date,'daily')
for col in pri.columns:
    if col.symbol in replace_prices:
        pri[col][-1] = replace_prices[col.symbol]

pos = 'pri[security]'
pos_one = (pri[SPY[0]][-1])
pos_six = (pri[SPY[0]][-75:].mean())
#VELOCITY
velocity_stop = (pos_one - pos_six)/100.0
SPY_Velocity = velocity_stop

net_income_positive_stocks = output[(output['net_income'] > 0)].index.tolist()

fulllongs = []
fullshorts = []
                
if SPY_Velocity > 0.0:
    fulllongs=output[(output['ret_quantile']==1) & 
                                (output['stdev_quantile']<nq_vol) & (output['gp_quantile']==nq_gp)].index.tolist()    
else:
    fulllongs=output[(output['ret_quantile']==1) & 
                                (output['stdev_quantile']<nq_vol) & (output['gp_quantile']==nq_gp) & (output['net_income'] > 0)].index.tolist()  
  
if SPY_Velocity > 0.0:
    fullshorts=output[(output['ret_quantile']==nq) &
                                 (output['stdev_quantile']<nq_vol) & (output['gp_quantile']==1) & (output['net_income'] < 0)].index.tolist() #(context.output['pb_quantile']==context.nq_pb) &   
else:
    fullshorts=output[(output['ret_quantile']==nq) &
                                 (output['stdev_quantile']<nq_vol) & (output['gp_quantile']==1)].index.tolist()   

fulllongs = fulllongs[-trade_count:]
    
if SPY_Velocity > 0.0:
    fullshorts = []
else:
    fullshorts = fullshorts[:trade_count]

portfolio_percent = {}    
    
total = len(fulllongs)
for security in fulllongs:
    portfolio_percent[security] = 1.0 / total
if SPY_Velocity <= 0:
    total = len(fullshorts)
    for security in fullshorts:
        portfolio_percent[security] = -0.7 / total    
    
#excludeshorts = []

#excludeshorts = [ 'ONCE','BLL']

positions = { 'VMW':[369,0], 'DBX':[2647,0], 'CDW':[412,0], 'AEP':[-369,0], 'BDX':[-144,0], 'D':[-423,0] } #liang
capital = 146900
#positions = {}

#shorts = [stock for stock in shorts if stock.symbol not in excludeshorts]
#fullshorts = [stock for stock in fullshorts if stock.-symbol not in excludeshorts]

trades = []

for pos in positions.keys():
    if pos in replace_prices:
        positions[pos][1] = replace_prices[pos]

positions2 = {}

for pos in positions.keys():
    positions2[symbols(pos)] = positions[pos]
        
positions = positions2

def trade_target_percent(security, percent):
    if security not in positions:
        positions[security] = [0,0]
        positions[security][1] = replace_prices[security.symbol]
        
    quantity = capital * percent / replace_prices[security.symbol]
    trade_quantity = quantity - positions[security][0]
    positions[security][0] = quantity
    
    if trade_quantity != 0:
        trades.append((security, trade_quantity, replace_prices[security.symbol], trade_quantity * replace_prices[security.symbol]))

for security in positions.keys():
    if security not in fulllongs and security not in fullshorts and security.symbol in replace_prices: 
        trade_target_percent(security, 0)
    else:
        if security.symbol in replace_prices:
            trade_target_percent(security, portfolio_percent[security])
    
for security in portfolio_percent.keys():
    if security.symbol in replace_prices:
        trade_target_percent(security, portfolio_percent[security])
            
trades

In [ ]:
[(Equity(1937 [CSX]), 254.0, 68.44, 17383.76),
 (Equity(49655 [TEAM]), -8.117582393421742, 151.61, -1230.7066666666703),
 (Equity(38936 [DG]), -133.0, 175.85, -23388.05),
 (Equity(3241 [GLW]), 2.1743777675981164, 21.83, 47.466666666666875),
 (Equity(44060 [ZTS]), -4.736499292597642, 124.87, -591.4466666666676),
 (Equity(4417 [LEN]), 347.0, 51.61, 17908.67),
 (Equity(2673 [F]), -3029.91452991453, 5.46, -16543.333333333332),
 (Equity(45667 [VEEV]), 126.56420143165711, 186.73, 23633.333333333332),
 (Equity(216 [HES]), -355.54122788165336, 46.53, -16543.333333333332)]
output[(output['ret_quantile']==1) & 
                                (output['stdev_quantile']<nq_vol) & (output['gp_quantile']==nq_gp)].index.tolist() 

In [ ]:
positions

In [ ]:
fulllongs